<a href="https://colab.research.google.com/github/denver-edwards/online-retail-customer/blob/main/Online_Retail_Customer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Online Retail Customer

**Author:** Denver Edwards

## Introduction and Objective
[Provide a brief description of the project and its goals and objectives]


In [ ]:
from google.colab import files
files.upload()

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d hassaneskikri/online-retail-customer-churn-dataset

import zipfile
zip_ref = zipfile.ZipFile('online-retail-customer-churn-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

Saving kaggle.json to kaggle.json
rm: cannot remove '/root/.kaggle': No such file or directory
  0% 0.00/23.2k [00:00<?, ?B/s]
100% 23.2k/23.2k [00:00<00:00, 38.1MB/s]


In [33]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

## Data Collection
[Code and description for loading the dataset and initial data exploration]


In [ ]:
# Code to load data
df = pd.read_csv("online_retail_customer_churn.csv")

# Initial exploration
print(df.info())
print(df.head())

# over_65 = len(df[df["Age"] > 65])
# print(over_65)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Customer_ID                 1000 non-null   int64  
 1   Age                         1000 non-null   int64  
 2   Gender                      1000 non-null   object 
 3   Annual_Income               1000 non-null   float64
 4   Total_Spend                 1000 non-null   float64
 5   Years_as_Customer           1000 non-null   int64  
 6   Num_of_Purchases            1000 non-null   int64  
 7   Average_Transaction_Amount  1000 non-null   float64
 8   Num_of_Returns              1000 non-null   int64  
 9   Num_of_Support_Contacts     1000 non-null   int64  
 10  Satisfaction_Score          1000 non-null   int64  
 11  Last_Purchase_Days_Ago      1000 non-null   int64  
 12  Email_Opt_In                1000 non-null   bool   
 13  Promotion_Response          1000 n

## Data Cleaning and Preprocessing
[Description of data cleaning and preprocessing steps]


In [ ]:
# Data cleaning and preprocessing steps
print("There are", df.isnull().sum().sum(), "missing values.")

There are 0 missing values.


## Exploratory Data Analysis (EDA)
[Description and code for statistical summary and data visualization]


In [ ]:
# EDA code
df.describe()

,Customer_ID,Age,Annual_Income,Total_Spend,Years_as_Customer,Num_of_Purchases,Average_Transaction_Amount,Num_of_Returns,Num_of_Support_Contacts,Satisfaction_Score,Last_Purchase_Days_Ago,Gender_Encoder
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,43.267000,111.962960,5080.79265,9.727000,49.456000,266.876530,4.612000,1.934000,2.974000,182.890000,0.982000
std,288.819436,15.242311,52.844111,2862.12335,5.536346,28.543595,145.873445,2.896869,1.402716,1.391855,104.391319,0.816298
min,1.000000,18.000000,20.010000,108.94000,1.000000,1.000000,10.460000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,250.750000,30.000000,67.800000,2678.67500,5.000000,25.000000,139.682500,2.000000,1.000000,2.000000,93.000000,0.000000
50%,500.500000,43.000000,114.140000,4986.19500,9.000000,49.000000,270.100000,5.000000,2.000000,3.000000,180.500000,1.000000
75%,750.250000,56.000000,158.452500,7606.47000,14.000000,74.000000,401.602500,7.000000,3.000000,4.000000,274.000000,2.000000
max,1000.000000,69.000000,199.730000,9999.64000,19.000000,99.000000,499.570000,9.000000,4.000000,5.000000,364.000000,2.000000


In [ ]:
# Gender Distribution
sns.countplot(x='Gender', data=df)
plt.title('Gender Distribution')
plt.show()

In [ ]:
# Churn Rate by Gender
sns.barplot(x='Gender', y='Target_Churn', data=df)
plt.title('Churn Rate by Gender')
plt.show()

In [ ]:
# Age Distribution by Gender
sns.boxplot(x='Gender', y='Age', data=df)
plt.title('Age Distribution by Gender')
plt.show()

In [ ]:
# Annual Income vs. Total Spend by Gender
sns.scatterplot(x='Annual_Income', y='Total_Spend', hue='Gender', data=df)
plt.title('Income vs. Spend by Gender')
plt.show()

In [ ]:
# Satisfaction Score by Gender
sns.boxplot(x='Gender', y='Satisfaction_Score', data=df)
plt.title('Customer Satisfaction by Gender')
plt.show()

## Feature Engineering and Selection
[Description of feature engineering and selection process]


In [ ]:
# Feature engineering and selection code
le_gender = LabelEncoder()
le_promotion_response = LabelEncoder()

df['Gender'] = le_gender.fit_transform(df["Gender"])
df["Promotion_Response"] = le_promotion_response.fit_transform(df["Promotion_Response"])

df["Return_Rate"] = df["Num_of_Returns"] / df["Num_of_Purchases"]
df["Avg_Spend_per_Purchase"] = df["Total_Spend"] / df["Num_of_Purchases"]
print(df)

## Model Building
[Description of model building process including splitting dataset and training models]


In [34]:
# Model training code
features = ["Return_Rate","Avg_Spend_per_Purchase","Promotion_Response"]
X = df[features]
y = df['Target_Churn']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 50, 100, 200]},
             scoring='accuracy', verbose=2)

## Model Evaluation
[Description of the model evaluation process and metrics]


In [35]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters:", best_params)
print("Best cross-validation score (accuracy):", best_score)

Best parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 10}
Best cross-validation score (accuracy): 0.54375


In [36]:
best_model = grid_search.best_estimator_

In [38]:
y_pred = best_model.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))

Accuracy on test set: 0.54


## Conclusion and Next Steps
[Summary of findings, conclusions, and recommendations for future work]


## References
[References or data sources used in the project]
